In [43]:
import pandas as pd

In [129]:
corpus = pd.read_csv('../data/google_ws_data_wn_synsets.csv', encoding='latin-1')
corpus = corpus.drop_duplicates()
corpus

,Unnamed: 0,lemma,pos,noad_sense,wn_sense,source,sentence
0,0,court,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us123...,Synset('court.n.02'),../data/word_sense_disambigation_corpora/masc/...,/ spoken / court transcript / Day3PMSession.tx...
1,1,court,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us123...,Synset('court.n.05'),../data/word_sense_disambigation_corpora/masc/...,/ spoken / court transcript / Day3PMSession.tx...
2,2,honorable,ADJ,/dictionary/sense/en_us_NOAD3e_2012/m_en_us125...,Synset('estimable.s.02'),../data/word_sense_disambigation_corpora/masc/...,"HONORABLE JOHN E. JONES , III Date :"
3,3,date,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us123...,Synset('date.n.08'),../data/word_sense_disambigation_corpora/masc/...,"HONORABLE JOHN E. JONES , III Date :"
4,4,date,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us123...,Synset('date.n.08'),../data/word_sense_disambigation_corpora/masc/...,"HONORABLE JOHN E. JONES , III Date :"
...,...,...,...,...,...,...,...
326902,41,period,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us127...,Synset('period.n.07'),../data/word_sense_disambigation_corpora/masc/...,/ written / newspaper : newswire / wsj_0127.tx...
326903,42,period,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us127...,Synset('period.n.03'),../data/word_sense_disambigation_corpora/masc/...,/ written / newspaper : newswire / wsj_0127.tx...
326904,43,time,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,Synset('prison_term.n.01'),../data/word_sense_disambigation_corpora/masc/...,/ written / newspaper : newswire / wsj_0127.tx...
326905,44,period,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us127...,Synset('period.n.07'),../data/word_sense_disambigation_corpora/masc/...,/ written / newspaper : newswire / wsj_0127.tx...


Now we see how many words we can use to compare to the experimental data, as a way to augment SEMCOR

In [130]:
#Cleaning our data from the FB JSON
import json
stimuli = json.load(open('../data/expt_stimuli.json', 'r'))
inputs = stimuli['inputs']
input_senses = [list(inputs[k].keys()) for k in inputs.keys()]
def wn_synset(fb_sense):
    parts = fb_sense.split('_')
    synset_str = '_'.join(parts[:len(parts) - 2]) + '.' + '.'.join(parts[-2:])
    return synset_str
expt_senses = [wn_synset(item) for sublist in input_senses for item in sublist if item not in ['type', 'senses']]

In [142]:
strip_synset = lambda s: s.strip("Synset('')")
corpus['wn_sense'] = corpus['wn_sense'].apply(strip_synset)
expt_words = corpus[corpus['wn_sense'].isin(expt_senses)][['lemma', 'pos', 'wn_sense', 'sentence']]
expt_words

,lemma,pos,wn_sense,sentence
108,case,NOUN,case.n.01,"Subsequently , the reporters were , if memory ..."
173,case,NOUN,case.n.01,"We 're brought here today by that , as well as..."
270,case,NOUN,case.n.01,I certainly understand the opinion and orders ...
311,indicate,VERB,indicate.v.02,"And what I mean by that , with regard to all o..."
312,indicate,VERB,indicate.v.03,"And what I mean by that , with regard to all o..."
...,...,...,...,...
326629,book,NOUN,book.n.01,"More shocking was the city of Hiroshima , whic..."
326630,book,NOUN,book.n.02,"More shocking was the city of Hiroshima , whic..."
326631,book,NOUN,record.n.05,"More shocking was the city of Hiroshima , whic..."
326649,ask,VERB,ask.v.01,"Someone woke him to say Gen. Hap Arnold , chie..."


In [153]:
shared_words = ['degree', 'table', 'foot', 'plane', 'right', 'model']
corpus[(corpus['lemma'].isin(shared_words)) & (corpus['pos'] == 'NOUN')].groupby('wn_sense').agg('count')

,Unnamed: 0,lemma,pos,noad_sense,source,sentence
wn_sense,,,,,,
able.n.01,171,171,171,171,171,171
academic_degree.n.01,6,6,6,6,6,6
airplane.n.01,12,12,12,12,12,12
board.n.04,20,20,20,20,20,20
degree.n.01,26,26,26,26,26,26
degree.n.07,27,27,27,27,27,27
foot.n.01,49,49,49,49,49,49
foot.n.11,222,222,222,222,222,222
model.n.01,38,38,38,38,38,38


In [156]:
corpus[(corpus['lemma'] == 'table') & (corpus['wn_sense'] == 'able.n.01')]

,Unnamed: 0,lemma,pos,noad_sense,wn_sense,source,sentence
13333,13333,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,"Yes , they did . Have you seen that book ? I h..."
13334,13334,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,"Yes , they did . Have you seen that book ? I h..."
13335,13335,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,"Yes , they did . Have you seen that book ? I h..."
28861,395,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,And the mother asked the little girl to please...
28862,396,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,And the mother asked the little girl to please...
...,...,...,...,...,...,...,...
282432,341,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,Translation : `` Why is n't it already on the ...
282433,342,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,Translation : `` Why is n't it already on the ...
322607,5081,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,But the Bush plan falls well short of the only...
322608,5082,table,NOUN,/dictionary/sense/en_us_NOAD3e_2012/m_en_us129...,able.n.01,../data/word_sense_disambigation_corpora/masc/...,But the Bush plan falls well short of the only...
